# To run this notebook:
1. Install all libraries
2. Update the directory of the training data in the config block
3. Uncomment lines related to kaggle and wandb if needed 
4. Run all blocks


# HW1: Frame-Level Speech Recognition

# Libraries

In [1]:
!pip install torchsummaryX==1.1.0 wandb --quiet

In [1]:
import torch
import numpy as np
from torchsummaryX import summary
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
from torch.cuda.amp import GradScaler, autocast
from datetime import datetime

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [2]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [6]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# !mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"username","key":"key"}')
#     # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

In [10]:
# # commands to download data from kaggle
#!kaggle competitions download -c 11785-hw1p2-f24

!unzip -qo /content/11785-hw1p2-f24.zip -d '/content'

unzip:  cannot find or open /content/11785-hw1p2-f24.zip, /content/11785-hw1p2-f24.zip.zip or /content/11785-hw1p2-f24.zip.ZIP.


# Dataset

In [3]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        # MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir = f'{root}/{partition}/mfcc'
        # Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = f'{root}/{partition}/transcript'

        # List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        # List files in self.transcript_dir using os.listdir in sorted order
        transcript_names = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        #  Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(f'{self.mfcc_dir}/{mfcc_names[i]}')

        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc_mean = np.mean(mfcc, axis=0)
            mfcc_std = np.std(mfcc, axis=0)
            mfcc = (mfcc - mfcc_mean) / mfcc_std

        #   Load the corresponding transcript
            transcript  = np.load(f'{self.transcript_dir}/{transcript_names[i]}')[1:-1] # Remove [SOS] and [EOS] from the transcript

        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)


        # Each mfcc is of shape T1 x 28, T2 x 28, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # the final shape is T x 28 (Where T = T1 + T2 + ...)
        self.mfccs = np.concatenate(self.mfccs, axis=0)

        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts, axis=0)

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), mode='constant')
        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

        # Map the phonemes to their corresponding list indexes in self.phonemes
        phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.array([phoneme_to_index[phoneme] for phoneme in self.transcripts])
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        frames = self.mfccs[ind:ind + 2 * self.context + 1]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten()

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [4]:
class AudioTestDataset(torch.utils.data.Dataset):

    def __init__(self, root, context=0, partition="test-clean"):
        self.context = context

        self.mfcc_dir = f'{root}/{partition}/mfcc'

        mfcc_names = sorted(os.listdir(self.mfcc_dir))

        self.mfccs = []

        for i in range(len(mfcc_names)):
            mfcc = np.load(f'{self.mfcc_dir}/{mfcc_names[i]}')

            # Cepstral Normalization
            mfcc_mean = np.mean(mfcc, axis=0)
            mfcc_std = np.std(mfcc, axis=0)
            mfcc = (mfcc - mfcc_mean) / mfcc_std

            self.mfccs.append(mfcc)

        self.mfccs = np.concatenate(self.mfccs, axis=0)
        self.length = len(self.mfccs)

        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), mode='constant')

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        frames = self.mfccs[ind:ind + 2 * self.context + 1].flatten()
        frames = torch.FloatTensor(frames)
        return frames

    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [9]:
config = {
    'epochs'        : 104,
    'batch_size'    : 4096,
    'context'       : 35,
    'init_lr'       : 0.006,
    'architecture'  : 'final',

    'scheduler': {
        'T_0': 7,
        'T_mult': 2,
        'eta_min': 1e-5,
        'type': 'CosineAnnealingWarnRestartsLR'
    },

    'weight_decay': 1e-3,
}

# TODO: Update this to point to actual data
root = "./content"

# Create Datasets

In [10]:
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(root, context=config['context'], partition="train-clean-100")

#TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(root, context=config['context'], partition="dev-clean")

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(root, context=config['context'], partition="test-clean")

In [11]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. 

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 12,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  4096
Context        :  35
Input size     :  1988
Output symbols :  42
Train dataset samples = 36091157, batches = 8812
Validation dataset samples = 1928204, batches = 471
Test dataset samples = 1934138, batches = 473


In [12]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([4096, 1988]) torch.Size([4096])


# Network Architecture


In [13]:
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),
            torch.nn.Dropout(0.35),


            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),
            torch.nn.Dropout(0.4),

            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),
            torch.nn.Dropout(0.35),


            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.GELU(),
            torch.nn.Dropout(0.4),


            torch.nn.Linear(2048, 1024),
            torch.nn.BatchNorm1d(1024),
            torch.nn.GELU(),
            torch.nn.Dropout(0.2),


            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.GELU(),


            torch.nn.Linear(512, 256),
            torch.nn.BatchNorm1d(256),
            torch.nn.GELU(),


            torch.nn.Linear(256, output_size)
        )

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [14]:
INPUT_SIZE  = (2*config['context'] + 1) * 28
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                [1988, 2048]         [4096, 2048]             4,073.47                 4.07
1_BatchNorm1d                 [2048]         [4096, 2048]                 4.10                 0.00
2_GELU                             -         [4096, 2048]                    -                    -
3_Dropout                          -         [4096, 2048]                    -                    -
4_Linear                [2048, 2048]         [4096, 2048]             4,196.35                 4.19
5_BatchNorm1d                 [2048]         [4096, 2048]                 4.10                 0.00
6_GELU                             -         [4096, 2048]                    -                    -
7_Dropout                          -         [4096, 2048]                    -                    -

In [27]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=config['init_lr'], weight_decay=config['weight_decay'])

scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=config['scheduler']['T_0'], eta_min=config['scheduler']['eta_min'], T_mult=config['scheduler']['T_mult'])

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [28]:
torch.cuda.empty_cache()
gc.collect()

451

In [29]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    scaler = GradScaler()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        with autocast():
          ### Forward Propagation
          logits  = model(frames)

          ### Loss Calculation
          loss    = criterion(logits, phonemes)

        ### Backward Propagation
        scaler.scale(loss).backward()

        ### Gradient Descent
        scaler.step(optimizer)
        scaler.update()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [30]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.



In [32]:
wandb.login(key="9042b018ca884dc92fa5a01ab8e447086b119f49") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


False

In [33]:
# # Create your wandb run
# run = wandb.init(
#     name    = "name", ### Wandb creates random run names if you skip this field, we recommend you give useful names
#     reinit  = True, ### Allows reinitalizing runs when you re-run this cell
#     #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
#     #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
#     project = "hw1p2", ### Project should be created in your wandb account
#     config  = config ### Wandb Config for your run
# )

run = wandb.init(
    name = "bs4096_lr6e-3",
    reinit=True,
    resume="allow",
    project="hw1p2",
    config = config,
)

In [34]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


['/home/msi/github/IDL_F24/HW1/P2/wandb/run-20251227_184508-75bvxp93/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [35]:
  # Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
# wandb.watch(model, log="all")

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    # scheduler.step(val_loss)
    scheduler.step()

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best



Epoch 1/104


/tmp/ipykernel_8720/420997.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Train:   0%|                                                                                                  …

/tmp/ipykernel_8720/420997.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Val:   0%|                                                                                                    …

...

Val:   0%|                                                                                                    …

	Train Acc 86.6803%	Train Loss 0.3766	 Learning Rate 0.0000523
	Val Acc 86.7684%	Val Loss 0.3916

Epoch 104/104


Train:   0%|                                                                                                  …

Val:   0%|                                                                                                    …

	Train Acc 86.6973%	Train Loss 0.3761	 Learning Rate 0.0000288
	Val Acc 86.7721%	Val Loss 0.3917


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [42]:
def test(model, test_loader):
    model.eval()

    ### List to store predicted phonemes of test data
    test_predictions = []

    with torch.no_grad(): 

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)


            # Get most likely predicted phoneme with argmax
            predicted_phoneme_ids = torch.argmax(logits, dim=1)

            # Map phoneme IDs to actual phonemes
            predicted_phonemes = [PHONEMES[id] for id in predicted_phoneme_ids.cpu().numpy()]

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            test_predictions.extend(predicted_phonemes)


    return test_predictions

In [43]:
predictions = test(model, test_loader)

  0%|          | 0/473 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x76cd8291e5f0><function _MultiProcessingDataLoaderIter.__del__ at 0x76cd8291e5f0>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/msi/miniconda3/envs/yolo/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
  File "/home/msi/miniconda3/envs/yolo/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()    
self._shutdown_workers()  File "/home/msi/miniconda3/envs/yolo/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers

  File "/home/msi/miniconda3/envs/yolo/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():    
  File "/home/msi/miniconda3/envs/yolo/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
if w.is_alive():    
assert self._parent_pid == os

In [38]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [39]:
### Finish your wandb run
run.finish()

lr,▇▅▄▂██▇▄▂█▆▆▅▄▄▂▂▁███▇▇▇▇▆▆▆▅▅▅▄▄▄▄▁▁▁▁▁
train_acc,▂▁▃▃▄▅▅▅▃▃▄▄▄▄▅▆▆▆▇▅▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇████
train_loss,█▆▅▅▇▃▃▃▅▄▄▄▄▃▃▃▂▂▂▂▄▄▄▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▃▅▅▆▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
valid_loss,█▃▃▃▃▂▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
lr,3e-05
train_acc,86.69726
train_loss,0.37606
val_acc,86.77208
valid_loss,0.39166


In [41]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
# !kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "From colab"

### However, its always safer to download the csv file and then upload to kaggle
torch.save(model.state_dict(), "model_20251227.pth")